In [ ]:
import warnings
warnings.filterwarnings('ignore')

from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/HD dacon2")
os.getcwd()

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import lightgbm as lgb
import bisect
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold

%matplotlib inline
plt.style.use(['dark_background'])
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',100)

In [ ]:
# 데이터 불러오기
train = pd.read_csv('train.csv').drop(columns=['SAMPLE_ID'])
test = pd.read_csv('test.csv').drop(columns=['SAMPLE_ID'])

In [ ]:
# 1. year, month, day, hour, minute, weekday 컬럼 그대로 생성
# datetime 컬럼 처리
train['ATA'] = pd.to_datetime(train['ATA'])
test['ATA'] = pd.to_datetime(test['ATA'])

# datetime을 여러 파생 변수로 변환
for df in [train, test]:
    df['year'] = df['ATA'].dt.year
    df['month'] = df['ATA'].dt.month
    df['day'] = df['ATA'].dt.day
    df['hour'] = df['ATA'].dt.hour
    df['minute'] = df['ATA'].dt.minute
    df['weekday'] = df['ATA'].dt.weekday


# 2. datetime 컬럼 제거
train.drop(columns='ATA', inplace=True)
test.drop(columns='ATA', inplace=True)


# 3. Categorical 컬럼 인코딩
categorical_features = ['ARI_CO', 'ARI_PO', 'SHIP_TYPE_CATEGORY', 'ID', 'SHIPMANAGER', 'FLAG']
encoders = {}

for feature in tqdm(categorical_features, desc="Encoding features"):
    le = LabelEncoder()
    train[feature] = le.fit_transform(train[feature].astype(str))
    le_classes_set = set(le.classes_)
    test[feature] = test[feature].map(lambda s: '-1' if s not in le_classes_set else s)
    le_classes = le.classes_.tolist()
    bisect.insort_left(le_classes, '-1')
    le.classes_ = np.array(le_classes)
    test[feature] = le.transform(test[feature].astype(str))
    encoders[feature] = le


In [ ]:
# 4. DIST가 0이면, CI_HOUR 모두 0으로 변경
train.loc[train['DIST'] == 0, 'CI_HOUR'] = 0

print(train)

In [ ]:
# 5. 결측치가 존재하는 데이터 1개 삭제
train = train.dropna(subset=['LENGTH']).reset_index(drop=True)

In [ ]:
# 6-1. 기상 관련 결측치 많은 컬럼 제거
train.drop(columns = 'U_WIND', inplace = True)
train.drop(columns = 'V_WIND', inplace = True)
train.drop(columns = 'AIR_TEMPERATURE', inplace = True)
train.drop(columns = 'BN', inplace = True)

test.drop(columns = 'U_WIND', inplace = True)
test.drop(columns = 'V_WIND', inplace = True)
test.drop(columns = 'AIR_TEMPERATURE', inplace = True)
test.drop(columns = 'BN', inplace = True)


In [1]:
# 6-2. 기상 관련 결측치 KNN을 통한 대체
'''
from sklearn.impute import KNNImputer

### 결측치 KNNImputer를 통해 대체

# K-NN 대체기
knn_imputer = KNNImputer(n_neighbors=10)

# K-NN 대체를 적용할 열을 지정
columns_to_impute = ['U_WIND', 'V_WIND', 'AIR_TEMPERATURE', 'BN']

# train 데이터프레임에서 결측치를 K-NN으로 대체
filtered_train[columns_to_impute] = knn_imputer.fit_transform(filtered_train[columns_to_impute])

filtered_train = pd.DataFrame(filtered_train, columns=filtered_train.columns)
#filtered_train.to_csv('train_imputed.csv', index = False)

'''

"\nfrom sklearn.impute import KNNImputer\n\n### 결측치 KNNImputer를 통해 대체\n\n# K-NN 대체기\nknn_imputer = KNNImputer(n_neighbors=10)\n\n# K-NN 대체를 적용할 열을 지정\ncolumns_to_impute = ['U_WIND', 'V_WIND', 'AIR_TEMPERATURE', 'BN']\n\n# train 데이터프레임에서 결측치를 K-NN으로 대체\nfiltered_train[columns_to_impute] = knn_imputer.fit_transform(filtered_train[columns_to_impute])\n\nfiltered_train = pd.DataFrame(filtered_train, columns=filtered_train.columns)\n#filtered_train.to_csv('train_imputed.csv', index = False)\n\n"

In [ ]:
# 데이터프레임 전체 결측치 파악
total_missing = train.isna().sum()
print(total_missing)

In [ ]:
# 7. 다중공선성 문제 해결을 위해 Length 컬럼 이외 삭제 (BREADTH, DEADWEIGHT, DEPTH, DRAUGHT, GT)

for df in [train, test]:
    df.drop(columns = 'BREADTH', inplace = True)
    df.drop(columns = 'DEADWEIGHT', inplace = True)
    df.drop(columns = 'DEPTH', inplace = True)
    df.drop(columns = 'DRAUGHT', inplace = True)
    df.drop(columns = 'GT', inplace = True)

In [ ]:
#  8. CI_HOUR 컬럼에서 표준편차 + 2 * 표준편차 이상이 되는 부분 이상치라고 판단 후, 삭제

# 'CI_HOUR' 열의 평균과 표준 편차 계산
mean = train['CI_HOUR'].mean()
std = train['CI_HOUR'].std()

# 이상치 경계 설정 (예: 평균에서 2배 표준 편차를 벗어나는 값)
lower_bound = mean - 2 * std
upper_bound = mean + 2 * std

# 이상치를 제거하고 정상 범위의 데이터만 남김
#filtered_train = train[(train['CI_HOUR'] >= lower_bound) & (train['CI_HOUR'] <= upper_bound)]
train = train[(train['CI_HOUR'] >= lower_bound) & (train['CI_HOUR'] <= upper_bound)].reset_index(drop=True)

In [ ]:
# Berting Time 고려한다면..?
'''
train['CI_HOUR'] = pd.to_timedelta(train['CI_HOUR'], unit="hours")
train['Berthing datetime'] = train['ATA'] + train['CI_HOUR']
test['CI_HOUR'] = pd.to_timedelta(test['CI_HOUR'], unit="hours")
test['Berthing datetime'] = test['ATA'] + test['CI_HOUR']
'''

In [ ]:
X_train = train.drop(columns='CI_HOUR')
y_train = train['CI_HOUR']